In [1]:
!pip install "kashgari>=1.1,<2.0"
!pip install "tensorflow>=1.14,<2.0"

In [2]:
folder_path = '.'
from sklearn.model_selection import train_test_split
from Preprocess import *
from output_format import *
import kashgari
import warnings
warnings.filterwarnings("ignore")

# Data Preprocess

In [3]:
#Train data
train_2 = f'{folder_path}/train_2.txt'
segment_text_list_2, BIO_list_2 = CRF_format(train_2)
x_train, y_train = split_sentence(segment_text_list_2, BIO_list_2, 80)

#Contest data
test = f'{folder_path}/test.txt'
test_data = loadInputFile_devData(test)
x_contest = [list(t) for t in test_data]
x_contest = split_sentence_tst(x_contest, 80)

print("Number of sentences :", len(x_train))
print("Number of words in first sentence :", len(x_train[0]))
print("Number of test article:",len(x_contest))

Number of sentences : 4855
Number of words in first sentence : 82
Number of test article: 159


In [4]:
x_train, y_train = delete_some_sentence(x_train, y_train)
x_train, x_valid, y_train, y_valid = train_test_split( x_train, y_train, test_size=0.2, random_state=24)
x_valid, x_test, y_valid, y_test = train_test_split( x_valid, y_valid, test_size=0.5, random_state=38)
print("Number of sentences in Training:", len(x_train))
print("Number of sentences in Validation:", len(x_valid))
print("Number of sentences in Test:", len(x_valid))

Number of sentences in Training: 2421
Number of sentences in Validation: 303
Number of sentences in Test: 303


In [5]:
print(x_train[0])
print(y_train[0])
print(x_valid[0])
print(y_valid[0])
print(x_test[0])
print(y_test[0])

['醫', '師', '：', '好', '，', '所', '以', '他', '是', '感', '染', '者', '嗎', '？', '還', '是', '不', '，', '不', '一', '定', '是', '？', '不', '是', '？', '民', '眾', '：', '我', '之', '前', '就', '是', '有', '跟', '他', '去', '驗', '過', '，', '啊', '他', '説', '不', '是', '，', '就', '…', '…', '醫', '師', '：', '就', '是', '去', '匿', '篩', '就', '是', '了', '？', '民', '眾', '：', '對', '。', '醫', '師', '：', '哦', '，', 'o', 'k', '。', '不', '過', '至', '少', '就', '是', '，']
['O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O']
['醫', '師', '：', '哦', '他', '要', '調', '整', '姿', '勢', '。', '民', '眾', '：', '對', '啊', '。', '醫', '師', '：', '超', '音', '波', '。', '民', '眾', '：', '啊', '他', '說', '，', '第', '二', '次', '要', '做

# Model Construct and Training

In [103]:
from kashgari.embeddings import BERTEmbedding
bert_embed = BERTEmbedding(f'{folder_path}/chinese_L-12_H-768_A-12', task=kashgari.LABELING, sequence_length=128)

In [91]:
from tensorflow.keras.callbacks import EarlyStopping, ModelCheckpoint,ReduceLROnPlateau
from kashgari.callbacks import EvalCallBack
from kashgari.tasks.labeling import BiLSTM_CRF_Model,BiLSTM_Model,CNN_LSTM_Model
model = BiLSTM_CRF_Model(bert_embed)
stop_callback = EarlyStopping(monitor='val_accuracy',patience=5, restore_best_weights=True)
eval_callback = EvalCallBack(kash_model=model,valid_x=x_valid,valid_y=y_valid,step=2)
reduce_lr = ReduceLROnPlateau(monitor='val_accuracy', factor=0.1, patience=2, verbose=1, min_lr=1e-6)
history = model.fit(x_train,
                    y_train,
                    epochs=20,
                    batch_size=24,
                    x_validate = x_valid, 
                    y_validate = y_valid, 
                    callbacks = [reduce_lr,eval_callback],
                    shuffle = True)
model.save(f'{folder_path}/BILSTMCRF_googlebert_batch24_sepwithmark_to80')

Model: "model_5"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
Input-Token (InputLayer)        [(None, 200)]        0                                            
__________________________________________________________________________________________________
Input-Segment (InputLayer)      [(None, 200)]        0                                            
__________________________________________________________________________________________________
Embedding-Token (TokenEmbedding [(None, 200, 768), ( 16226304    Input-Token[0][0]                
__________________________________________________________________________________________________
Embedding-Segment (Embedding)   (None, 200, 768)     1536        Input-Segment[0][0]              
____________________________________________________________________________________________

In [92]:
model.evaluate(x_test,y_test)

                precision    recall  f1-score   support

      location     0.8378    0.8158    0.8267        38
        family     1.0000    0.1250    0.2222         8
          time     0.7744    0.8548    0.8126       241
      med_exam     0.8269    0.9149    0.8687        47
          name     0.8537    0.8140    0.8333        43
         money     0.8333    0.9091    0.8696        11
       contact     0.0000    0.0000    0.0000         5
    profession     0.0000    0.0000    0.0000         9
            ID     0.0000    0.0000    0.0000         5
clinical_event     0.0000    0.0000    0.0000         1

     micro avg     0.7951    0.7990    0.7971       408
     macro avg     0.7628    0.7990    0.7727       408



'                precision    recall  f1-score   support\n\n      location     0.8378    0.8158    0.8267        38\n        family     1.0000    0.1250    0.2222         8\n          time     0.7744    0.8548    0.8126       241\n      med_exam     0.8269    0.9149    0.8687        47\n          name     0.8537    0.8140    0.8333        43\n         money     0.8333    0.9091    0.8696        11\n       contact     0.0000    0.0000    0.0000         5\n    profession     0.0000    0.0000    0.0000         9\n            ID     0.0000    0.0000    0.0000         5\nclinical_event     0.0000    0.0000    0.0000         1\n\n     micro avg     0.7951    0.7990    0.7971       408\n     macro avg     0.7628    0.7990    0.7727       408\n'

# Make Contest Output

In [6]:
# Choose trained model
model_name = "/BILSTMCRF_googlebert_batch24_sepwithmark_to80"
#model_name = "/BILSTMCRF_googlebert_batch36_sepwithmark_to80"
#model_name = "/BILSTMCRF_roberta_wwm_large_ext_L-24_H-1024_A-16_batch32_sepwithmark_to80"
#model_name = "/BILSTMCRF_wwm_ext_bert_batch24_sepwithmark_to80"
path = folder_path + model_name
model = kashgari.utils.load_model(f'{path}')

In [7]:
make_contest_output(x_contest, model, path)

Finish Predict article  0
Finish Predict article  1
Finish Predict article  2
Finish Predict article  3
Finish Predict article  4
Finish Predict article  5
Finish Predict article  6
Finish Predict article  7
Finish Predict article  8
Finish Predict article  9
Finish Predict article  10
Finish Predict article  11
Finish Predict article  12
Finish Predict article  13
Finish Predict article  14
Finish Predict article  15
Finish Predict article  16
Finish Predict article  17
Finish Predict article  18
Finish Predict article  19
Finish Predict article  20
Finish Predict article  21
Finish Predict article  22
Finish Predict article  23
Finish Predict article  24
Finish Predict article  25
Finish Predict article  26
Finish Predict article  27
Finish Predict article  28
Finish Predict article  29
Finish Predict article  30
Finish Predict article  31
Finish Predict article  32
Finish Predict article  33
Finish Predict article  34
Finish Predict article  35
Finish Predict article  36
Finish Pred